# Combining and analysing the low density anglescan simulations for all 5 probe tips

In [1]:
%matplotlib tk
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import xarray as xr
import pandas as pd
import scipy.io as sio
import sys
import os
import glob
import copy
import pathlib as pth
import importlib
sys.path.append('/home/jleland/Coding/Projects/flopter')
import flopter.spice.splopter as spl
import flopter.spice.tdata as td
import flopter.spice.homogenise as hmg
import flopter.core.ivdata as iv
import flopter.core.fitters as fts
import flopter.core.fitdata as fd
import flopter.core.lputils as lpu

In [2]:
importlib.reload(spl)
importlib.reload(td)
importlib.reload(hmg)
importlib.reload(fd)
importlib.reload(lpu)

<module 'flopter.core.lputils' from '/home/jleland/Coding/Projects/flopter/flopter/core/lputils.py'>

## Paths and Matfile IO
This section deals with file io and selecting the right .mat files. This needs to be run for the latter sections to work.

In [2]:
# lowdens_dir = pth.Path('/home/jleland/Spice/spice2/bin/data_local_m2/lowdens_anglescan')
lowdens_dir = pth.Path('/home/jleland/Data/external/spice/')
os.chdir(lowdens_dir)

In [3]:
lps = lpu.MagnumProbes()

flush_probe = copy.deepcopy(lps.probe_l)
flush_probe.theta_p = 0.0
flush_probe.d_perp = 0.0

angled_probe = copy.deepcopy(lps.probe_l)
angled_probe.d_perp = 0.0
print(angled_probe.theta_p)

0.17453292519943295


In [4]:
skippable_scans = {
    'marconi/lowdens_anglescan/inpartest_as',
    'marconi/lowdens_anglescan/angled_tiltscan_ext'
}
single_sims = {
    'marconi/lowdens_anglescan/angled_1',
}

In [5]:
scans_searchstr = '*/lowdens_anglescan/*'
# angles_search_str = '/*[!.{yml, inp}]/backup*'
angles_search_str = '/*[!.{yml, inp}]'

non_standard_variables = {'ProbePot', 'npartproc', 'Nz', 'Nzmax', 'Ny', 'count', 'Npc', 'snumber', 'nproc'}

desired_variables = td.DEFAULT_REDUCED_DATASET | non_standard_variables

all_run_dirs = {}
scans = glob.glob(scans_searchstr)
scans = set(scans) - skippable_scans
for scan in scans:
    if scan in single_sims:
        print(f'Single sim {scan} found')
        all_run_dirs[scan] = [scan]
    else:
        all_run_dirs[scan] = glob.glob(scan + angles_search_str)
    
    print(f'{scan}: {all_run_dirs[scan]}\n')


Single sim marconi/lowdens_anglescan/angled_1 found
marconi/lowdens_anglescan/angled_1: ['marconi/lowdens_anglescan/angled_1']

marconi/lowdens_anglescan/rearwall_tiltscan: ['marconi/lowdens_anglescan/rearwall_tiltscan/alpha_yz_-1.0', 'marconi/lowdens_anglescan/rearwall_tiltscan/alpha_yz_-10.0', 'marconi/lowdens_anglescan/rearwall_tiltscan/alpha_yz_-15.0', 'marconi/lowdens_anglescan/rearwall_tiltscan/alpha_yz_-2.0', 'marconi/lowdens_anglescan/rearwall_tiltscan/alpha_yz_-20.0', 'marconi/lowdens_anglescan/rearwall_tiltscan/alpha_yz_-3.0', 'marconi/lowdens_anglescan/rearwall_tiltscan/alpha_yz_-30.0', 'marconi/lowdens_anglescan/rearwall_tiltscan/alpha_yz_-4.0', 'marconi/lowdens_anglescan/rearwall_tiltscan/alpha_yz_-45.0', 'marconi/lowdens_anglescan/rearwall_tiltscan/alpha_yz_-5.0', 'marconi/lowdens_anglescan/rearwall_tiltscan/alpha_yz_-6.0', 'marconi/lowdens_anglescan/rearwall_tiltscan/alpha_yz_-60.0', 'marconi/lowdens_anglescan/rearwall_tiltscan/alpha_yz_-7.0', 'marconi/lowdens_anglescan/

## Plotting all IV characterisitics on the same axes

In [16]:
skippable = {
    'marconi/lowdens_anglescan/angled_tiltscan/alpha_yz_-1.0',
    'marconi/lowdens_anglescan/angled_tiltscan/alpha_yz_-2.0',
    'marconi/lowdens_anglescan/angled_tiltscan/alpha_yz_-3.0',
    'marconi/lowdens_anglescan/inpartest_as_restart/alpha_yz_-1.0',
    'marconi/lowdens_anglescan/inpartest_as_restart/alpha_yz_-2.0',
    'marconi/lowdens_anglescan/inpartest_as_restart/alpha_yz_-3.0',
    'cumulus/lowdens_anglescan/angled_tiltscan_redo/alpha_yz_-1.0',
    'cumulus/lowdens_anglescan/angled_tiltscan_redo/alpha_yz_-1.0_restart',
    'cumulus/lowdens_anglescan/angled_tiltscan_redo/alpha_yz_-90.0',
    'cumulus/lowdens_anglescan/flush_tiltscan_redo/alpha_yz_-90.0',
    'cumulus/lowdens_anglescan/flush_tiltscan_redo/alpha_yz_-1.0',
    'cumulus/lowdens_anglescan/angled_tiltscan_ext/alpha_yz_-1.0',
    'marconi/lowdens_anglescan/inpartest_as/alpha_yz_-1.0',
    'marconi/lowdens_anglescan/inpartest_as/alpha_yz_-2.0',
    'marconi/lowdens_anglescan/inpartest_as/alpha_yz_-3.0',
    'marconi/lowdens_anglescan/inpartest_as/alpha_yz_-4.0',
    'marconi/lowdens_anglescan/inpartest_as/alpha_yz_-5.0',
    'marconi/lowdens_anglescan/inpartest_as/alpha_yz_-6.0',
    'marconi/lowdens_anglescan/inpartest_as/alpha_yz_-7.0',
    'marconi/lowdens_anglescan/inpartest_as/alpha_yz_-8.0',
    'marconi/lowdens_anglescan/inpartest_as/alpha_yz_-9.0',
    'marconi/lowdens_anglescan/recessed_tiltscan/alpha_yz_-1.0',
    'marconi/lowdens_anglescan/recessed_tiltscan/alpha_yz_-2.0',
    'marconi/lowdens_anglescan/recessed_tiltscan/alpha_yz_-3.0',
    'marconi/lowdens_anglescan/recessed_tiltscan/alpha_yz_-4.0',
    'marconi/lowdens_anglescan/recessed_tiltscan/alpha_yz_-5.0',
    'marconi/lowdens_anglescan/rearwall_tiltscan/alpha_yz_-1.0',
    'marconi/lowdens_anglescan/rearwall_tiltscan/alpha_yz_-2.0',
    'marconi/lowdens_anglescan/rearwall_tiltscan/alpha_yz_-3.0',
    'marconi/lowdens_anglescan/rearwall_tiltscan/alpha_yz_-4.0',
    'marconi/lowdens_anglescan/rearwall_tiltscan/alpha_yz_-5.0',
    'marconi/lowdens_anglescan/sprobe_tiltscan/alpha_yz_-1.0',
    'marconi/lowdens_anglescan/sprobe_tiltscan/alpha_yz_-10.0',
#     'cumulus/lowdens_anglescan/flush_tiltscan_ext/alpha_yz_-1.0',
#     'cumulus/lowdens_anglescan/angled_tiltscan_ext/alpha_yz_-1.0'
}

In [17]:
# fig, axes = plt.subplots(2)
splopters = {}
scans = list(scans)
for i in range(len(scans)):
#     ax = axes[i]
    scan = scans[i]

    for angle_dir in all_run_dirs[scan]:
        print(f'\nCreating splopter for {scan}, {angle_dir}')
        if angle_dir in skippable:
            print('Skipping...')
            continue
        try:
            splopter = spl.Splopter(lowdens_dir / angle_dir, reduce=desired_variables)
            splopter.prepare(denormaliser_fl=True, homogenise_fl=True, find_se_temp_fl=False)
    #         splopter.denormalise()
            splopters[angle_dir] = splopter
        except AssertionError as e:
            print(f'AssertionError encountered for {angle_dir}. Skipping...')
            continue
        
print('\n...Done')


Creating splopter for marconi/lowdens_anglescan/angled_1, marconi/lowdens_anglescan/angled_1

Creating splopter for marconi/lowdens_anglescan/rearwall_tiltscan, marconi/lowdens_anglescan/rearwall_tiltscan/alpha_yz_-1.0
Skipping...

Creating splopter for marconi/lowdens_anglescan/rearwall_tiltscan, marconi/lowdens_anglescan/rearwall_tiltscan/alpha_yz_-10.0
Looking for a suitable backup to use instead.
AssertionError encountered for marconi/lowdens_anglescan/rearwall_tiltscan/alpha_yz_-10.0. Skipping...

Creating splopter for marconi/lowdens_anglescan/rearwall_tiltscan, marconi/lowdens_anglescan/rearwall_tiltscan/alpha_yz_-15.0
Looking for a suitable backup to use instead.
AssertionError encountered for marconi/lowdens_anglescan/rearwall_tiltscan/alpha_yz_-15.0. Skipping...

Creating splopter for marconi/lowdens_anglescan/rearwall_tiltscan, marconi/lowdens_anglescan/rearwall_tiltscan/alpha_yz_-2.0
Skipping...

Creating splopter for marconi/lowdens_anglescan/rearwall_tiltscan, marconi/lo

/home/jleland/Coding/Environments/python3.6/lib/python3.6/site-packages/numpy/core/_methods.py:217: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims)
/home/jleland/Coding/Environments/python3.6/lib/python3.6/site-packages/numpy/core/_methods.py:186: RuntimeWarning: invalid value encountered in true_divide
  arrmean, rcount, out=arrmean, casting='unsafe', subok=False)
/home/jleland/Coding/Environments/python3.6/lib/python3.6/site-packages/numpy/core/_methods.py:209: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)



Creating splopter for cumulus/lowdens_anglescan/recessed_tiltscan_redo, cumulus/lowdens_anglescan/recessed_tiltscan_redo/alpha_yz_-1.0

Creating splopter for cumulus/lowdens_anglescan/recessed_tiltscan_redo, cumulus/lowdens_anglescan/recessed_tiltscan_redo/alpha_yz_-2.0

Creating splopter for cumulus/lowdens_anglescan/recessed_tiltscan_redo, cumulus/lowdens_anglescan/recessed_tiltscan_redo/alpha_yz_-3.0

Creating splopter for cumulus/lowdens_anglescan/recessed_tiltscan_redo, cumulus/lowdens_anglescan/recessed_tiltscan_redo/alpha_yz_-4.0

Creating splopter for cumulus/lowdens_anglescan/recessed_tiltscan_redo, cumulus/lowdens_anglescan/recessed_tiltscan_redo/alpha_yz_-5.0

Creating splopter for cumulus/lowdens_anglescan/angled_tiltscan_ext, cumulus/lowdens_anglescan/angled_tiltscan_ext/alpha_yz_-1.0
Skipping...

Creating splopter for cumulus/lowdens_anglescan/angled_tiltscan_ext, cumulus/lowdens_anglescan/angled_tiltscan_ext/alpha_yz_-70.0

Creating splopter for cumulus/lowdens_anglesca

In [40]:
for i, (filename, splopter) in enumerate(splopters.items()):
    print(f'{i}) {filename}: {splopter}')

0) cumulus/lowdens_anglescan/angled_tiltscan_redo/alpha_yz_-10.0: <flopter.spice.splopter.Splopter object at 0x7f6641939ef0>
1) cumulus/lowdens_anglescan/angled_tiltscan_redo/alpha_yz_-15.0: <flopter.spice.splopter.Splopter object at 0x7f6641900048>
2) cumulus/lowdens_anglescan/angled_tiltscan_redo/alpha_yz_-2.0: <flopter.spice.splopter.Splopter object at 0x7f6641900f98>
3) cumulus/lowdens_anglescan/angled_tiltscan_redo/alpha_yz_-20.0: <flopter.spice.splopter.Splopter object at 0x7f663092d2b0>
4) cumulus/lowdens_anglescan/angled_tiltscan_redo/alpha_yz_-25.0: <flopter.spice.splopter.Splopter object at 0x7f66417077b8>
5) cumulus/lowdens_anglescan/angled_tiltscan_redo/alpha_yz_-3.0: <flopter.spice.splopter.Splopter object at 0x7f66308cd898>
6) cumulus/lowdens_anglescan/angled_tiltscan_redo/alpha_yz_-30.0: <flopter.spice.splopter.Splopter object at 0x7f662e55a358>
7) cumulus/lowdens_anglescan/angled_tiltscan_redo/alpha_yz_-45.0: <flopter.spice.splopter.Splopter object at 0x7f662e1e5898>
8)

In [11]:
# For adding a new run to the list of splopters
angle_dir = all_run_dirs[scans[6]][0]
print(angle_dir)

if angle_dir in splopters:
    print(f'Removing {angle_dir} from existing list of splopters')
    splopters.pop(angle_dir)
splopter = spl.Splopter(lowdens_dir / angle_dir, reduce=desired_variables)
splopter.prepare(denormaliser_fl=True, homogenise_fl=True, find_se_temp_fl=False)
splopter.denormalise()
splopters[angle_dir] = splopter

marconi/lowdens_anglescan/inpartest_as_restart/alpha_yz_-1.0


## Fitting and Adding Info to a Dataframe
Might be good to make it into a dataset instead so individual IV chracterisitcs can be stored as well.

In [18]:
fig, axes = plt.subplots(4, 3, sharex='col', sharey='col')

fit_df = pd.DataFrame(columns=['probe', 
                               'angle', 
                               'temp',
                               'd_temp',
                               'isat', 
                               'd_isat',
                               'a',
                               'd_a',
                               'v_f',
                               'd_v_f'
                              ])

for label, s in splopters.items():
    print('=== ' + label + ' ===')
#     if angle == 2.0:
#         continue
    
    if label in skippable:
        print(f'Skipping {label}')
        continue
    elif 'inpartest' in label or 'flush' in label:
        ax = axes[0][0]
        axt = axes[0][1]
        axd = axes[0][2]
        probe = 'flush'
        ax.set_title('Flush')
    elif 'angled' in label:
        ax = axes[1][0]
        axt = axes[1][1]
        axd = axes[1][2]
        probe = 'angled'
        ax.set_title('Angled Tip')
    elif 'rearwall' in label:
        ax = axes[2][0]
        axt = axes[2][1]
        axd = axes[2][2]
        probe = 'rearwall'
        ax.set_title('Rearwall Tip (L)')
    elif 'recessed' in label:
        ax = axes[3][0]
        axt = axes[3][1]
        axd = axes[3][2]
        probe = 'recessed'
        ax.set_title('Recessed Tip')
    else:
        print(f'Probe not matching expected categories, skipping...')
        continue
        
#     angle = np.abs(float(label[-4:]))
    angle = np.degrees(np.abs(np.squeeze(s.tdata.alphayz)))
    print(angle)

    
    iv_region = np.where(s.iv_data['V'] < -2)
    i = np.where((np.diff(s.iv_data['V'][iv_region]) == 0))
    i = (i[0] + 1, )
    
#     if len(i[0]) / len(iv_region[0]) <= 0.25:
#         print('Big downsample, skipping...')
    i = iv_region
#     s.denormalise()
    
    ax.errorbar(s.iv_data['V'][i], s.iv_data['I'][i], label=angle, yerr=s.iv_data['sigma'][i])
    ax.legend()
    
    try:
        fit_iv_data = iv.IVData.non_contiguous_trim(s.iv_data, i)
        
        if any(len(data_arr) == 0 for variable, data_arr in fit_iv_data.items()):
            print(f'Found empty data array in {label}')
            continue
        fitter = fts.FullIVFitter()
        fit_data = fitter.fit_iv_data(fit_iv_data)
        #     ax.plot(*fit_data.get_fit_plottables())

        fit_df = fit_df.append({'probe': probe, 
                                'angle': angle, 
                                'temp': fit_data.get_temp(),
                                'd_temp': fit_data.get_temp_err(),
                                'isat': fit_data.get_isat(), 
                                'd_isat': fit_data.get_isat_err(),
                                'a': fit_data.get_sheath_exp(), 
                                'd_a': fit_data.get_sheath_exp_err(),
                                'v_f': fit_data.get_floating_pot(),
                                'd_v_f': fit_data.get_floating_pot_err()}, ignore_index=True)

        axt.errorbar(angle, fit_data.get_temp(), yerr=fit_data.get_temp_err(), fmt='x') #, ecolor='silver')
        axd.errorbar(angle, fit_data.get_isat(), yerr=fit_data.get_isat_err(), fmt='x') #, ecolor='silver')
    except RuntimeError as e:
        print(f'WARNING: Encountered error in fit, skipping {label}')

for j, probe in enumerate(['flush', 'angled']):
    filt_df = fit_df.where(fit_df.probe == probe).dropna()
    axt = axes[j][1]
    axd = axes[j][2]
    if j == 0:
        axt.set_title(r'$T_e$')
        axd.set_title(r'$I_{sat}$')
    if j == 1:
        axt.set_xlabel(r'$\theta$')
        axd.set_xlabel(r'$\theta$')
#     axt.errorbar(filt_df['angle'], filt_df['temp'] - 1, yerr=filt_df['d_temp'], fmt='x', ecolor='silver')
#     axt.set_ylim(0.3, 1.7)
    axt.axhline(y=1.0, color='k', linewidth=0.5, linestyle='dotted')
#     axd.errorbar(filt_df['angle'], filt_df['isat'], yerr=filt_df['d_isat'], fmt='x', ecolor='silver')
#     axd.set_ylim(-0.1, 90)
    
fit_df


=== marconi/lowdens_anglescan/angled_1 ===
1.0
=== cumulus/lowdens_anglescan/flush_tiltscan_redo/alpha_yz_-10.0 ===
10.0
=== cumulus/lowdens_anglescan/flush_tiltscan_redo/alpha_yz_-15.0 ===
14.999999999999998
=== cumulus/lowdens_anglescan/flush_tiltscan_redo/alpha_yz_-2.0 ===
2.0
=== cumulus/lowdens_anglescan/flush_tiltscan_redo/alpha_yz_-20.0 ===
20.0
=== cumulus/lowdens_anglescan/flush_tiltscan_redo/alpha_yz_-25.0 ===
25.0
=== cumulus/lowdens_anglescan/flush_tiltscan_redo/alpha_yz_-3.0 ===
3.0000000000000004
=== cumulus/lowdens_anglescan/flush_tiltscan_redo/alpha_yz_-30.0 ===
29.999999999999996
=== cumulus/lowdens_anglescan/flush_tiltscan_redo/alpha_yz_-45.0 ===
45.0
=== cumulus/lowdens_anglescan/flush_tiltscan_redo/alpha_yz_-60.0 ===
59.99999999999999
=== cumulus/lowdens_anglescan/flush_tiltscan_redo/alpha_yz_-75.0 ===
75.0
=== marconi/lowdens_anglescan/inpartest_as_restart/alpha_yz_-4.0 ===
4.0
=== marconi/lowdens_anglescan/inpartest_as_restart/alpha_yz_-5.0 ===
5.0
=== marconi/low

,probe,angle,temp,d_temp,isat,d_isat,a,d_a,v_f,d_v_f
0,angled,1.0,1.025535,0.020026,3.283136,0.082634,3.440025e-02,0.008213,-3.372417,0.007165
1,flush,10.0,1.011952,0.009473,23.709109,0.223009,5.236428e-02,0.002896,-2.981223,0.002314
2,flush,15.0,1.055231,0.012629,36.369258,0.426445,3.041129e-02,0.003559,-2.932875,0.002739
3,flush,2.0,0.746410,0.041318,3.856090,0.219110,1.897804e-01,0.018324,-2.920495,0.018052
4,flush,20.0,1.035289,0.010473,46.674624,0.443865,3.084954e-02,0.002835,-2.867274,0.002253
5,flush,25.0,1.054621,0.027365,57.868608,1.363051,2.583858e-02,0.006994,-2.819666,0.005345
6,flush,3.0,1.512113,0.236044,10.719348,1.784801,8.417440e-15,0.056576,-3.020600,0.026887
7,flush,30.0,1.020983,0.015933,67.124746,0.950988,2.801778e-02,0.004150,-2.817061,0.003384
8,flush,45.0,1.013971,0.011028,95.638252,0.896645,1.896112e-02,0.002690,-2.759501,0.002245
9,flush,60.0,1.034100,0.017558,115.508637,1.624039,1.063754e-02,0.004007,-2.673119,0.003366


In [19]:
fit_df = fit_df.sort_values(['probe', 'angle']).reset_index(drop=True)
fit_df

,probe,angle,temp,d_temp,isat,d_isat,a,d_a,v_f,d_v_f
0,angled,1.0,1.025535,0.020026,3.283136,0.082634,3.440025e-02,0.008213,-3.372417,0.007165
1,angled,2.0,1.045895,0.016761,7.148155,0.163483,1.398953e-02,0.007510,-3.526380,0.006954
2,angled,3.0,1.048542,0.008816,11.100458,0.129524,4.755583e-03,0.003758,-3.484159,0.003447
3,angled,4.0,1.005186,0.017391,14.159134,0.326872,1.674877e-02,0.007322,-3.373216,0.006888
4,angled,5.0,1.010071,0.010163,17.942595,0.250309,1.249651e-02,0.004458,-3.457913,0.004251
5,angled,6.0,0.979680,0.006385,20.553665,0.167514,2.640946e-02,0.002550,-3.281570,0.002369
6,angled,7.0,0.930221,0.006718,23.710272,0.169637,2.973546e-02,0.002070,-2.968275,0.001954
7,angled,8.0,0.888347,0.016929,26.718170,0.539216,3.050686e-02,0.005812,-3.014594,0.005967
8,angled,9.0,1.048546,0.008319,32.368312,0.329622,1.167446e-02,0.003248,-3.362969,0.002814
9,angled,10.0,1.033696,0.010810,35.128063,0.458634,1.811731e-02,0.004147,-3.335452,0.003589


In [20]:
fit_data_filename = 'lowdens_fitdata.csv'

In [21]:
# Save dataframe
fit_df.to_csv(fit_data_filename)

## Loading and using the dataframe


In [ ]:
fit_data_filename = 'lowdens_fitdata.csv'